## Compare likelihoods for theories with different values of H_0

We redo the notebook full_likelihood_example, but now varying also H_0

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 140
mpl.rcParams['figure.dpi'] = 140
# our modules
import data_MPGADGET
import p1d_arxiv
import gp_emulator
import likelihood
import likelihood_parameter
import lya_theory
import full_theory
import camb_cosmo
import CAMB_model
import fit_linP

### Setup mock data, from one of simulations in the suite

In [2]:
# specify simulation suite and P1D mesurements
basedir="/p1d_emulator/sim_suites/Australia20/"
skewers_label='Ns500_wM0.05'
p1d_label=None
# specify simulation to use to generate synthetic data
test_sim_num=15
data=data_MPGADGET.P1D_MPGADGET(sim_number=test_sim_num,
                                basedir=basedir,
                                skewers_label=skewers_label,
                                data_cov_factor=1)

Note: redshifts have been re-sorted (earliest first)
Note: redshifts have been re-sorted (earliest first)


### Setup archive of P1D measurements from simulations and train the emulator

In [3]:
# specify redshift range
z_max=4
# do not use test_sim_num that was used in generating mock data
arxiv=p1d_arxiv.ArxivP1D(basedir=basedir,drop_sim_number=test_sim_num,
                            drop_tau_rescalings=True,z_max=z_max,
                            drop_temp_rescalings=True,skewers_label=skewers_label)

In [4]:
# specify parameters to be used in emulator
paramList=['mF', 'sigT_Mpc', 'gamma', 'kF_Mpc', 'Delta2_p', 'n_p']
# specify k range
kmax_Mpc=8
# setup GP emulator
emu=gp_emulator.GPEmulator(basedir,p1d_label,skewers_label,z_max=z_max,
                                verbose=False,paramList=paramList,train=True,
                                emu_type="k_bin",passArxiv=arxiv,kmax_Mpc=kmax_Mpc,
                                drop_tau_rescalings=True,
                                drop_temp_rescalings=True)

 /Users/font/Programs/Others/anaconda3/lib/python3.6/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


Training GP on 270 points
Optimised


### Setup and compare theory objects

Start by setting up an old theory object, that uses only compressed linP parameters

In [5]:
old_theory=lya_theory.LyaTheory(zs=data.z,emulator=emu,verbose=True)

Note: redshifts have been re-sorted (earliest first)
Note: redshifts have been re-sorted (earliest first)


Setup now a new theory object, that holds a CAMB object

In [6]:
new_theory=full_theory.FullTheory(zs=data.z,emulator=emu,verbose=True)

In [7]:
# compare emulator calls at one of the redshifts
old_emu_calls=old_theory.get_emulator_calls()
new_emu_calls=new_theory.get_emulator_calls(return_M_of_z=False)
print("z=2 emulator calls for old theory = ", old_emu_calls[0])
print("z=2 emulator calls for new theory = ", new_emu_calls[0])

0 2.0 model {'Delta2_p': 0.6300728196477859, 'n_p': -2.2994793242105556, 'alpha_p': -0.2163759500235969, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
1 2.25 model {'Delta2_p': 0.5400906361802847, 'n_p': -2.2994795007686575, 'alpha_p': -0.21637592419496002, 'f_p': 0.9666989020097663, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.13051263566195997, 'kF_Mpc': 9.574544756852742}
2 2.5 model {'Delta2_p': 0.46777658657004356, 'n_p': -2.299479573535976, 'alpha_p': -0.21637620821376063, 'f_p': 0.9729433890617769, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12986720978610813, 'kF_Mpc': 9.91377946463774}
3 2.75 model {'Delta2_p': 0.40887628967130535, 'n_p': -2.299479391313876, 'alpha_p': -0.21637623273087048, 'f_p': 0.9776963475296402, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.1290634736772292, 'kF_Mpc': 10.258134266642541}
4 3.

Note that the numbers above are not supposed to exactly match, since kp_Mpc is different in both theories

### Compare now emulator calls for a different test cosmology

In [8]:
As=1.9e-9
ns=0.92
H0=74.0
test_cosmo=camb_cosmo.get_cosmology(H0=H0,As=As,ns=ns)
test_camb_model=CAMB_model.CAMBModel(zs=data.z,cosmo=test_cosmo)

In [9]:
# print compressed linP parameters from the fiducial cosmology in the old theory
old_linP=fit_linP.LinearPowerModel(cosmo=old_theory.cosmo.cosmo_fid)
old_like_params=old_linP.get_likelihood_parameters()
for par in old_like_params:
    print(par.info_str())

g_star = 0.9677508579459803
f_star = 0.981369557840318
Delta2_star = 0.36017590528641974
n_star = -2.299335667257498
alpha_star = -0.2165270371206895


In [10]:
# print compressed linP parameters from the test cosmology
test_linP=fit_linP.LinearPowerModel(cosmo=test_cosmo)
old_test_like_params=test_linP.get_likelihood_parameters()
for par in old_test_like_params:
    print(par.info_str())

g_star = 0.9577005905767146
f_star = 0.975786292900187
Delta2_star = 0.2894527099942636
n_star = -2.3454640097099224
alpha_star = -0.21599697156493478


In [11]:
# we want to study also cases where we ignore some of the parameters
#cull_params=[]
cull_params=['f_star','g_star','alpha_star']
culled_like_params = [par for par in old_test_like_params if par.name not in cull_params]
for par in culled_like_params:
    print(par.info_str())

Delta2_star = 0.2894527099942636
n_star = -2.3454640097099224


In [12]:
# call the old theory, but using the freshly computed likelihood parameters from the test cosmology
old_test_emu_calls=old_theory.get_emulator_calls(like_params=culled_like_params)

0 2.0 model {'Delta2_p': 0.506353374737127, 'n_p': -2.3456076666629793, 'alpha_p': -0.21637595002359608, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
1 2.25 model {'Delta2_p': 0.4340398566100403, 'n_p': -2.345607843221083, 'alpha_p': -0.21637592419496718, 'f_p': 0.9666989020097663, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.13051263566195997, 'kF_Mpc': 9.574544756852742}
2 2.5 model {'Delta2_p': 0.3759252039552537, 'n_p': -2.3456079159884013, 'alpha_p': -0.216376208213755, 'f_p': 0.9729433890617769, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12986720978610813, 'kF_Mpc': 9.91377946463774}
3 2.75 model {'Delta2_p': 0.32859041474094175, 'n_p': -2.3456077337662995, 'alpha_p': -0.21637623273085152, 'f_p': 0.9776963475296402, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.1290634736772292, 'kF_Mpc': 10.258134266642541}
4 3.0 

In [13]:
# print cosmology parameters from the new theory for the test cosmology
new_like_params=new_theory.camb_model_fid.get_likelihood_parameters()
for par in new_like_params:
    print(par.info_str())

ombh2 = 0.022
omch2 = 0.12
As = 2.1e-09
ns = 0.965
H0 = 67.0


In [14]:
# print cosmology parameters from new test camb model
new_test_like_params=test_camb_model.get_likelihood_parameters()
for par in new_test_like_params:
    print(par.info_str())

ombh2 = 0.022
omch2 = 0.12
As = 1.9e-09
ns = 0.92
H0 = 74.0


In [15]:
# call the new theory, but using the freshly computed likelihood parameters for the test cosmology
new_test_emu_calls=new_theory.get_emulator_calls(like_params=new_test_like_params,return_M_of_z=False)

Note: redshifts have been re-sorted (earliest first)
0 2.0 model {'Delta2_p': 0.5021783618635486, 'n_p': -2.3444376892101615, 'alpha_p': -0.21653624850999953, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.12925597769550834, 'kF_Mpc': 9.34396201809978}
1 2.25 model {'Delta2_p': 0.4312217302557556, 'n_p': -2.344437429549661, 'alpha_p': -0.2165363865906534, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.12928815573631092, 'kF_Mpc': 9.665224663184658}
2 2.5 model {'Delta2_p': 0.3739805313296343, 'n_p': -2.3444376202100665, 'alpha_p': -0.21653553050168303, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12887745943253898, 'kF_Mpc': 9.989915095907442}
3 2.75 model {'Delta2_p': 0.32722238678901944, 'n_p': -2.3444375145668244, 'alpha_p': -0.2165355951371306, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.12825491819176194, 'kF_Mpc': 10.322804462911757}
4 3.0 model {'Delta2_p': 0.2885870803396723, 'n_p': -2.3444

In [16]:
print("z=2 emulator calls for old test theory = ", old_test_emu_calls[0])
print("z=2 emulator calls for new test theory = ", new_test_emu_calls[0])

z=2 emulator calls for old test theory =  {'Delta2_p': 0.506353374737127, 'n_p': -2.3456076666629793, 'alpha_p': -0.21637595002359608, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
z=2 emulator calls for new test theory =  {'Delta2_p': 0.5021783618635486, 'n_p': -2.3444376892101615, 'alpha_p': -0.21653624850999953, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.12925597769550834, 'kF_Mpc': 9.34396201809978}


In [17]:
print("z = {} emulator calls for old test theory = {}".format(data.z[-1],old_test_emu_calls[-1]))
print("z = {} emulator calls for new test theory = {}".format(data.z[-1],new_test_emu_calls[-1]))

z = 4.0 emulator calls for old test theory = {'Delta2_p': 0.18648788535182184, 'n_p': -2.3456082035193075, 'alpha_p': -0.21637634322605842, 'f_p': 0.9898232760729386, 'mF': 0.3667549106286143, 'gamma': 1.4366944565860078, 'sigT_Mpc': 0.1124868806234996, 'kF_Mpc': 12.841201989686333}
z = 4.0 emulator calls for new test theory = {'Delta2_p': 0.18619752156110378, 'n_p': -2.3444374213597414, 'alpha_p': -0.21653545860707465, 'mF': 0.3667549106286143, 'gamma': 1.4366944565860078, 'sigT_Mpc': 0.1121811457105948, 'kF_Mpc': 12.876198991607117}


Both new and old theories make the same emulator call for a theory that is not the fiducial!

### Test the likelihood objects with old and new theories

In [18]:
## Set up a likelihood object with the old theory object
#old_free_params=['Delta2_star','n_star']
old_free_params=['Delta2_star','n_star','alpha_star','f_star','g_star']
old_like=likelihood.Likelihood(data=data,theory=old_theory,
                            free_parameters=old_free_params,
                            free_param_limits=None,
                            verbose=True,
                            prior_Gauss_rms=0.15,
                            emu_cov_factor=1)

got parameters
g_star = 0.9677508579459803
f_star = 0.981369557840318
Delta2_star = 0.36017590528641974
n_star = -2.299335667257498
alpha_star = -0.2165270371206895
ln_tau_0 = 0.0
ln_tau_1 = 0.0
ln_sigT_kms_0 = 0.0
ln_sigT_kms_1 = 0.0
ln_gamma_0 = 0.0
ln_gamma_1 = 0.0
ln_kF_0 = 0.0
ln_kF_1 = 0.0
likelihood setup with 5 free parameters
5 free parameters


In [19]:
## Set up a likelihood object with the new theory object
new_free_params=["As","ns","H0"]
new_like=likelihood.Likelihood(data=data,theory=new_theory,
                            free_parameters=new_free_params,
                            free_param_limits=None,
                            verbose=True,
                            prior_Gauss_rms=0.15,
                            emu_cov_factor=1)

got parameters
ombh2 = 0.022
omch2 = 0.12
As = 2.1e-09
ns = 0.965
H0 = 67.0
ln_tau_0 = 0.0
ln_tau_1 = 0.0
ln_sigT_kms_0 = 0.0
ln_sigT_kms_1 = 0.0
ln_gamma_0 = 0.0
ln_gamma_1 = 0.0
ln_kF_0 = 0.0
ln_kF_1 = 0.0
likelihood setup with 3 free parameters
3 free parameters


In [20]:
# figure out values of parameters in unit cube of old theory
old_values=[]
for parname in old_free_params:
    for p in old_test_like_params:
        if p.name==parname:
            old_values.append(p.value_in_cube())
            print(p.info_str(),p.value_in_cube())

Delta2_star = 0.2894527099942636 0.2630180666284239
n_star = -2.3454640097099224 0.0453599029007767
alpha_star = -0.21599697156493478 0.49093662213695666
f_star = 0.975786292900187 0.6446573225046764
g_star = 0.9577005905767146 0.1925147644178669


In [21]:
old_like.get_log_like(values=old_values,ignore_log_det_cov=False)

0 2.0 model {'Delta2_p': 0.5578241265239452, 'n_p': -2.2856779247025907, 'alpha_p': -0.24867228881692963, 'f_p': 0.9288017519441485, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13043246903562247, 'kF_Mpc': 9.259680163451705}
1 2.25 model {'Delta2_p': 0.4799412663998917, 'n_p': -2.285678101260692, 'alpha_p': -0.24867226298830422, 'f_p': 0.9371437402854793, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.13024777998621095, 'kF_Mpc': 9.594014359496517}
2 2.5 model {'Delta2_p': 0.417226443102854, 'n_p': -2.285678174028011, 'alpha_p': -0.24867254700709937, 'f_p': 0.94338822733749, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12969139360101506, 'kF_Mpc': 9.927219083389073}
3 2.75 model {'Delta2_p': 0.36604475297168165, 'n_p': -2.2856779918059087, 'alpha_p': -0.24867257152421923, 'f_p': 0.9481411858053532, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.12897605046227628, 'kF_Mpc': 10.265087488374748}
4 3.0

-134.19018600182727

In [22]:
# figure out values of parameters in unit cube of new theory
new_values=[]
for parname in new_free_params:
    for p in new_test_like_params:
        if p.name==parname:
            new_values.append(p.value_in_cube())
            print(p.info_str(),p.value_in_cube())

As = 1.9e-09 -0.500000000000001
ns = 0.92 -0.9999999999999963
H0 = 74.0 0.7857142857142857


In [23]:
new_like.get_log_like(values=new_values,ignore_log_det_cov=False)

Note: redshifts have been re-sorted (earliest first)
0 2.0 model {'Delta2_p': 0.5021783618635486, 'n_p': -2.3444376892101615, 'alpha_p': -0.21653624850999953, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.12925597769550834, 'kF_Mpc': 9.34396201809978}
1 2.25 model {'Delta2_p': 0.4312217302557556, 'n_p': -2.344437429549661, 'alpha_p': -0.2165363865906534, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.12928815573631092, 'kF_Mpc': 9.665224663184658}
2 2.5 model {'Delta2_p': 0.3739805313296343, 'n_p': -2.3444376202100665, 'alpha_p': -0.21653553050168303, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12887745943253898, 'kF_Mpc': 9.989915095907442}
3 2.75 model {'Delta2_p': 0.32722238678901944, 'n_p': -2.3444375145668244, 'alpha_p': -0.2165355951371306, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.12825491819176194, 'kF_Mpc': 10.322804462911757}
4 3.0 model {'Delta2_p': 0.2885870803396723, 'n_p': -2.3444

47.513571180382584